In [5]:
# pdf plumber
import requests
from io import BytesIO
import pdfplumber

# data analysis
import numpy as np
import pandas as pd
import re
import csv

import scraper

In [6]:
smic_urls_file = 'urls/smic_urls.csv'
smic_urls_reader = csv.reader(open( 'urls/smic_urls.csv', 'r', encoding='utf-8-sig'))
smic_urls = dict([(row[0] + " " + row[1], row[2]) for row in smic_urls_reader])

In [7]:
for item in smic_urls.items():
    print(item[0])
    if item[1] not in ('NA', 'Link'):
        scraper.parse_smic(url)

2004 1
geo broken
segment broken
firstColName
DEBUG: successful
2004 2
geo broken
segment broken
firstColName
DEBUG: successful
2004 3
geo broken
segment broken
firstColName
DEBUG: successful
2004 4
geo broken
segment broken
firstColName
DEBUG: successful
2005 1
geo broken
segment broken
firstColName
DEBUG: successful
2005 2
geo broken
segment broken
firstColName
DEBUG: successful
2005 3
geo broken
segment broken
firstColName
DEBUG: successful
2005 4
geo broken
segment broken
firstColName
DEBUG: successful
2006 1
geo broken
segment broken
firstColName
DEBUG: successful
2006 2
geo broken
segment broken
firstColName
DEBUG: successful
2006 3
geo broken
segment broken
firstColName
DEBUG: successful
2006 4
geo broken
segment broken
firstColName
DEBUG: successful
2007 1
geo broken
segment broken
firstColName
DEBUG: successful
2007 2
geo broken
segment broken
firstColName
DEBUG: successful
2007 3
geo broken
segment broken
firstColName
DEBUG: successful
2007 4
geo broken
segment broken
firstCo

In [8]:
url = "https://smic.shwebspace.com/uploads/qe/201208090828548_en.pdf"
rq = requests.get(url)
pdf = pdfplumber.open(BytesIO(rq.content))
smic_text = ""
for i in range(5):
    smic_text += pdf.pages[i].extract_text()

In [20]:
def clean_smic_tech(smic_text):
    digits = "\s+(\d+(?:\.\d+)?%)\s+(\d+(?:\.\d+)?%)\s+(\d+(?:\.\d+)?%)"
    quarters = "\s+(\d+Q\d+)\s+(\d+Q\d+)\s+(\d+Q\d+)"
    segCols = np.asarray(re.findall(f"(By (?:Customer|Service) Type){quarters}", smic_text)[0])
    fabless = np.asarray(re.findall(f"(Fabless).*?{digits}", smic_text)[0])
    idm = np.asarray(re.findall(f"(Integrated device manufacturers).*?{digits}", smic_text)[0])
    systemComp = np.asarray(re.findall(f"(System companies and (?:O|o)thers){digits}", smic_text)[0])
    tsmcPlat = np.array([fabless, idm, systemComp])
    platDF = pd.DataFrame(tsmcPlat, columns=segCols)
    return platDF

In [26]:
def clean_smic_seg(smic_text):
    digits = "\s+(\d+(?:\.\d+)?%)\s+(\d+(?:\.\d+)?%)\s+(\d+(?:\.\d+)?%)"
    quarters = "\s+(\d+Q\d+)\s+(\d+Q\d+)\s+(\d+Q\d+)"
    segCols = np.asarray(re.findall(f"(By (?:{quarters})?(?:Service|Customer) Type).*?(?:{quarters})?", smic_text)[0])
    fabless = np.asarray(re.findall(f"(Fabless).*?{digits}", smic_text)[0])
    idm = np.asarray(re.findall(f"(Integrated device manufacturers).*?{digits}", smic_text)[0])
    systemComp = np.asarray(re.findall(f"(System companies and others).*?{digits}", smic_text)[0])
    tsmcPlat = np.array([fabless, idm, systemComp])
    platDF = pd.DataFrame(tsmcPlat, columns=segCols)
    return platDF

In [27]:
def clean_smic_tech_robust(smic_text):
    digits = "\s+(\dQ\d+)\s+(\dQ\d+)\s+(\dQ\d+)"
    oldHeader = "Total\s+(?:\\n)?Wafer\s+(?:\\n)?Revenues"
    try:
        techCols = np.asarray(re.findall(f"(By Technology|{oldHeader}){digits}", smic_text)[0])
    except: 
        techCols = np.array("Tech")
        techCols = np.append(techCols, np.asarray(re.findall(f"{digits}", smic_text)[0]))
    techArray = []
    digit_re = "\d+(?:\.\d+)(?:\s+)?(?:n|µ|m)m"
    slash = "\d+(?:\.\d+)?\/\d+(?:\.\d+)?\s+(?:n|µ|m)m"
    above = "\d+(?:\.\d+)?(?:u|n|m)m\s+and\s+above"
    numbers = "\s+(\d+(?:\.\d+)?%)\s+(\d+(?:\.\d+)?%)\s+(\d+(?:\.\d+)?%)"
    techRegex = f"({digit_re}|{slash}|{above}){numbers}"
    for i in re.finditer(techRegex, smic_text):
        techArray.append(list(i.groups()))
    techDF = pd.DataFrame(techArray, columns=techCols)
    firstColName = techDF.columns[0]
    print("firstColName")
    techDF = techDF.drop_duplicates(subset=firstColName, keep="first")
    return techDF

In [28]:
clean_smic_seg(smic_text)

IndexError: list index out of range

In [40]:
smic_text

' \n \n \nContact:            \n \nInvestor Relations \n        \n+86-21-3861-0000 ext. 12804 \nir@smics.com   \n \nNEWS RELEASE \n \nSMIC Reports 2012 Second Quarter Results \n \nAll currency figures stated in this report are in US Dollars unless stated otherwise. \nThe financial statement amounts in this report are determined in accordance with US GAAP. \n \nShanghai, China –August 8, 2012.  Semiconductor Manufacturing International Corporation (NYSE: \nSMI; SEHK: 981) (“SMIC” or the “Company”), one of the leading semiconductor foundries in the world, \ntoday announced its consolidated results of operations for the three months ended June 30, 2012.   \n \nSecond Quarter 2012 Highlights:  \n \n(cid:1)  Record-high quarterly revenue of $421.8 million in 2Q12, which was up by 26.8% from \n$332.7 million in 1Q12 and jumped by 19.7% year over year. \n(cid:1)  Gross margin doubled to 24.1% in 2Q12 compared to 12.0% in 1Q12, primarily due to a \nhigher utilization rate and continued improve

In [43]:
scraper.parse_smic("https://smic.shwebspace.com/uploads/qe/201208090828548_en.pdf")

geo broken
segment broken
firstColName
DEBUG: successful
